In [2]:
import pandas as pd
import json
import unidecode
import random
import pickle

relation_list = ["http://dbpedia.org/ontology/populationTotal",
                "http://dbpedia.org/ontology/genre",
                "http://dbpedia.org/property/timezone",
                "http://dbpedia.org/ontology/timeZone",
                "http://dbpedia.org/ontology/birthPlace",
                "http://dbpedia.org/ontology/location",
                "http://dbpedia.org/property/mapCaption",
                "http://dbpedia.org/property/location",
                "http://dbpedia.org/ontology/isPartOf",
                "http://dbpedia.org/ontology/position",
                "http://dbpedia.org/ontology/deathPlace",
                "http://dbpedia.org/ontology/writer",
                "http://dbpedia.org/ontology/artist",
                "http://dbpedia.org/ontology/country",
                "http://dbpedia.org/ontology/recordLabel",
                "http://dbpedia.org/ontology/literaryGenre",
                "http://dbpedia.org/ontology/type",
                "http://dbpedia.org/ontology/director",
                "http://dbpedia.org/ontology/language",
                "http://dbpedia.org/ontology/hometown",
                "http://dbpedia.org/ontology/producer",
                "http://dbpedia.org/ontology/author",
                
                "http://dbpedia.org/ontology/deathCause",
                "http://dbpedia.org/ontology/instrument",
                "http://dbpedia.org/property/modes",
                "http://dbpedia.org/ontology/musicComposer",
                "http://dbpedia.org/ontology/publisher",
                "http://dbpedia.org/ontology/battle",
                "http://dbpedia.org/ontology/nationality",
                "http://dbpedia.org/ontology/influencedBy",
                "http://dbpedia.org/ontology/place",
                "http://dbpedia.org/ontology/industry"
                'http://dbpedia.org/ontology/creator',
               'http://dbpedia.org/ontology/developer',
               'http://dbpedia.org/ontology/religion',
               'http://dbpedia.org/ontology/occupation',
               'http://dbpedia.org/ontology/architecturalStyle',
               'http://dbpedia.org/ontology/parent',
               'http://dbpedia.org/ontology/locatedInArea',
               'http://dbpedia.org/ontology/cinematography',
               'http://dbpedia.org/ontology/distributor',
               'http://dbpedia.org/ontology/child',
               'http://dbpedia.org/ontology/restingPlace',
               'http://dbpedia.org/property/genre',
               'http://dbpedia.org/ontology/field',
               'http://dbpedia.org/ontology/headquarter',
               'http://dbpedia.org/ontology/ideology',
               'http://dbpedia.org/property/religion',
               'http://dbpedia.org/property/birthPlace',
               'http://dbpedia.org/ontology/foundedBy',
               'http://dbpedia.org/ontology/series',
                
               'http://dbpedia.org/ontology/city', 
               'http://dbpedia.org/ontology/owner',
               'http://dbpedia.org/property/areas',
               'http://dbpedia.org/ontology/region',
               'http://dbpedia.org/ontology/influenced',
               'http://dbpedia.org/ontology/mountainRange',
               'http://dbpedia.org/ontology/languageFamily',
               'http://dbpedia.org/property/namedFor',
               'http://dbpedia.org/ontology/movement',
               'http://dbpedia.org/ontology/spokenIn',
               'http://dbpedia.org/ontology/broadcastArea',
               'http://dbpedia.org/ontology/state',
               'http://dbpedia.org/ontology/architect',
               'http://dbpedia.org/property/deathPlace',
               'http://dbpedia.org/property/weight',
               'http://dbpedia.org/property/constell',
               'http://dbpedia.org/ontology/ethnicity',
               'http://dbpedia.org/ontology/crosses',
               'http://dbpedia.org/ontology/license',
               'http://dbpedia.org/ontology/manufacturer',
               'http://dbpedia.org/ontology/foundationPlace',
               'http://dbpedia.org/ontology/hubAirport',
               'http://dbpedia.org/ontology/programmingLanguage',
               'http://dbpedia.org/ontology/doctoralAdvisor',
               'http://dbpedia.org/property/position',
               'http://dbpedia.org/ontology/successor',
               'http://dbpedia.org/property/currentpositionplain',
               'http://dbpedia.org/ontology/riverMouth',
               'http://dbpedia.org/ontology/outflow',
               'http://dbpedia.org/property/governingBody',
               'http://dbpedia.org/ontology/sourceCountry',
               'http://dbpedia.org/property/writer',
               'http://dbpedia.org/ontology/musicalArtist',
               'http://dbpedia.org/ontology/era',
               'http://dbpedia.org/ontology/citizenship',
               'http://dbpedia.org/ontology/notableWork',
               'http://dbpedia.org/ontology/subsequentWork',
               'http://dbpedia.org/property/architectureStyle',
               'http://dbpedia.org/property/productionCompanies',
               'http://dbpedia.org/property/playerPositions',
               'http://dbpedia.org/property/publishingCountry',
               'http://dbpedia.org/property/namedafter',
               'http://dbpedia.org/ontology/executiveProducer',
               'http://dbpedia.org/ontology/coach',
               'http://dbpedia.org/ontology/province',
               'http://dbpedia.org/property/members',
               'http://dbpedia.org/property/park']

In [3]:
#train_df[train_df.predicate == 'http://dbpedia.org/ontology/producer']

In [4]:
with open('../data/SimpleQuestionsDBpedia/train.json') as json_file:
    train = json.load(json_file)
    
with open('../data/SimpleQuestionsDBpedia/test.json') as json_file:
    test = json.load(json_file)
    
with open('../data/SimpleQuestionsDBpedia/valid.json') as json_file:
    valid = json.load(json_file)    

In [7]:
def get_questions_and_predicates(data):
    questions, predicates = list(), list()

    for question in data['Questions']:
        text = unidecode.unidecode(question['Query'])
        predicate = question['PredicateList'][0]['Predicate']

        questions.append(text)
        predicates.append(predicate)
    
    return questions, predicates

In [8]:
train_questions, train_predicates = get_questions_and_predicates(train)
test_questions, test_predicates = get_questions_and_predicates(test)
val_questions, val_predicates = get_questions_and_predicates(valid)

In [9]:
train_df = pd.DataFrame.from_dict({'question': train_questions, 'predicate': train_predicates})
test_df = pd.DataFrame.from_dict({'question': test_questions, 'predicate': test_predicates})
val_df = pd.DataFrame.from_dict({'question': val_questions, 'predicate': val_predicates})

In [12]:
train_df.predicate.value_counts().keys()

Index(['http://dbpedia.org/ontology/genre',
       'http://dbpedia.org/ontology/birthPlace',
       'http://purl.org/linguistics/gold/hypernym',
       'http://dbpedia.org/ontology/isPartOf',
       'http://xmlns.com/foaf/0.1/gender',
       'http://dbpedia.org/ontology/position',
       'http://dbpedia.org/ontology/deathPlace',
       'http://dbpedia.org/ontology/writer',
       'http://dbpedia.org/ontology/artist',
       'http://dbpedia.org/ontology/country',
       'http://dbpedia.org/ontology/recordLabel',
       'http://dbpedia.org/ontology/literaryGenre',
       'http://dbpedia.org/ontology/type',
       'http://dbpedia.org/ontology/director',
       'http://dbpedia.org/ontology/language',
       'http://dbpedia.org/ontology/location',
       'http://dbpedia.org/ontology/timeZone',
       'http://dbpedia.org/ontology/hometown',
       'http://dbpedia.org/ontology/producer',
       'http://dbpedia.org/ontology/author',
       'http://dbpedia.org/ontology/deathCause',
       'http

In [26]:
genre = train_df[train_df['predicate'].isin(['http://dbpedia.org/ontology/genre'])].sample(600)
birth_place = train_df[train_df['predicate'].isin(['http://dbpedia.org/ontology/birthPlace'])].sample(600)

In [27]:
genre.shape

(600, 2)

In [28]:
train_df.head()

,predicate,question
0,http://dbpedia.org/ontology/distributor,what movie is produced by warner bros.
1,http://purl.org/linguistics/gold/hypernym,What is don graham known as?
2,http://dbpedia.org/ontology/location,what's there to see in columbus
3,http://dbpedia.org/ontology/birthPlace,who is a musician born in detroit
4,http://dbpedia.org/ontology/hometown,Which city did the artist ryna originate in


In [29]:
train_df = train_df[train_df['predicate'].isin(relation_list)]
test_df = test_df[test_df['predicate'].isin(relation_list)]
#train_df = train_df.append(test_df).append(genre).append(birth_place)

val_df = val_df[val_df['predicate'].isin(relation_list)]

In [30]:
train_df['predicate'].value_counts()

http://dbpedia.org/ontology/genre                 4919
http://dbpedia.org/ontology/birthPlace            3577
http://dbpedia.org/ontology/isPartOf              1730
http://dbpedia.org/ontology/position              1254
http://dbpedia.org/ontology/deathPlace            1045
http://dbpedia.org/ontology/writer                 996
http://dbpedia.org/ontology/artist                 994
http://dbpedia.org/ontology/country                889
http://dbpedia.org/ontology/recordLabel            776
http://dbpedia.org/ontology/literaryGenre          685
http://dbpedia.org/ontology/type                   590
http://dbpedia.org/ontology/director               494
http://dbpedia.org/ontology/language               464
http://dbpedia.org/ontology/timeZone               411
http://dbpedia.org/ontology/location               411
http://dbpedia.org/ontology/hometown               407
http://dbpedia.org/ontology/producer               403
http://dbpedia.org/ontology/author                 347
http://dbp

In [31]:
len(val_df['predicate'].value_counts())

45

In [32]:
train_df.shape

(24147, 2)

In [33]:
test_df.shape

(6868, 2)

In [34]:
import nltk

nltk.download('wordnet')
nltk.download('stopwords')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')

[nltk_data] Downloading package wordnet to /home/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
def preprocess_text(text, remove_stopwords=False):
    # clean_ascii
    tmp = "".join(i for i in text if ord(i) < 128)
    #lowercase
    tmp = tmp.lower()
    #normal form
    tokens = tokenizer.tokenize(tmp)
    #stopwords
    if remove_stopwords:
        prep_text = ' '.join(t for t in tokens if t not in stopwords)
    else:
        prep_text = ' '.join(t for t in tokens)
    
    return prep_text

In [36]:
%%time

train_df['clean text'] = train_df['question'].apply(lambda x: preprocess_text(x))
#test_df['clean text'] = test_df['question'].apply(lambda x: preprocess_text(x))
val_df['clean text'] = val_df['question'].apply(lambda x: preprocess_text(x))

CPU times: user 465 ms, sys: 0 ns, total: 465 ms
Wall time: 463 ms


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

seed = 42

In [38]:
vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,1))
vectorizer.fit(train_df.append(val_df)['clean text'])

pickle.dump(vectorizer, open("relation_vectorizer.model", 'wb'))

In [39]:
encoder = LabelEncoder()
encoder.fit(train_df.append(val_df)['predicate'])

pickle.dump(encoder, open("relation_encoder.model", 'wb'))

In [40]:
print("Total number of unique n_grams in data = {0}".format(len(list(vectorizer.vocabulary_.keys()))))

Total number of unique n_grams in data = 4142


In [41]:
X_train = vectorizer.transform(train_df['clean text'])
y_train = encoder.transform(train_df['predicate'])

In [42]:
classifier = LogisticRegression(random_state=seed).fit(X_train, y_train)

pickle.dump(classifier, open("relation_classifier.model", 'wb'))

/home/alex/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [43]:
X_test = vectorizer.transform(val_df['clean text'])
y_test = encoder.transform(val_df['predicate'])

In [44]:
y_pred = classifier.predict(X_test)

In [46]:
print(classification_report(y_test, y_pred,target_names = encoder.inverse_transform([i for i in range(45)])))

                                                precision    recall  f1-score   support

http://dbpedia.org/ontology/architecturalStyle       1.00      0.94      0.97        18
            http://dbpedia.org/ontology/artist       0.88      0.91      0.89       137
            http://dbpedia.org/ontology/author       0.91      0.68      0.78        60
            http://dbpedia.org/ontology/battle       1.00      1.00      1.00        41
        http://dbpedia.org/ontology/birthPlace       0.85      0.97      0.90       482
             http://dbpedia.org/ontology/child       0.71      0.42      0.53        12
    http://dbpedia.org/ontology/cinematography       1.00      0.88      0.93         8
           http://dbpedia.org/ontology/country       0.72      0.43      0.54       134
        http://dbpedia.org/ontology/deathCause       1.00      0.96      0.98        27
        http://dbpedia.org/ontology/deathPlace       0.99      0.99      0.99       143
         http://dbpedia.org/ont

/home/alex/miniconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
text = preprocess_text("was angela merkel born in hamburg?")
vector = vectorizer.transform([text])
prediction = classifier.predict(vector)
probas = classifier.predict_proba(vector)
class_ = encoder.inverse_transform(prediction)

print(class_)

['http://dbpedia.org/ontology/birthPlace']


In [48]:
probas

array([[2.29216861e-04, 4.27152656e-04, 3.67074805e-04, 1.09702382e-03,
        9.71466839e-01, 1.78534047e-04, 3.04235233e-04, 2.17086370e-03,
        4.90405453e-04, 2.40490677e-03, 3.15095545e-04, 3.94738716e-04,
        2.29077817e-04, 1.75050818e-04, 1.75256998e-04, 7.93267691e-04,
        2.76965128e-04, 1.12024907e-03, 1.31762252e-04, 4.76429654e-04,
        2.59706385e-04, 7.81609085e-04, 7.24223988e-04, 3.83794843e-04,
        4.49419869e-04, 1.04351389e-03, 4.48068649e-04, 1.74742175e-03,
        1.80097249e-04, 2.99549084e-04, 7.53252189e-04, 5.96121820e-04,
        4.22339179e-04, 3.23941276e-04, 3.58186292e-04, 2.51019468e-04,
        5.07939667e-04, 1.44390571e-04, 3.63609799e-04, 5.28228575e-04,
        6.35517548e-04, 4.85881851e-03, 1.65288829e-04, 4.02463386e-04,
        1.47331681e-04]])

## Query template Classifier (BOOL, DISTANCE, FORWARD/BACKWARD)

In [17]:
distance = pd.read_csv("../data/ManuallyGeneratedData/distance.intent", sep='\t')
cities = pd.read_csv("../data/ManuallyGeneratedData/world-cities.csv")

distance_templates = distance.question.values
cities = list(cities.name.values)

distance_questions = list()

for i in range(15):
    for template in distance_templates:
        city_1 = random.choice(cities)
        cities.remove(city_1)
        city_2 = random.choice(cities)
        cities.remove(city_2)

        template = template.replace("X", unidecode.unidecode(city_1))
        template = template.replace("Y", unidecode.unidecode(city_2))

        distance_questions.append(template)

In [18]:
lc_quad_train = pd.read_csv("../data/LC-QuAD/train-data.csv", sep=';')
lc_quad_test = pd.read_csv("../data/LC-QuAD/test-data.csv", sep=';')

lc_quad = lc_quad_train.append(lc_quad_test)
lc_quad = lc_quad[lc_quad['class'] == 'BOOL']

In [19]:
lc_quad['class'].value_counts()

BOOL    368
Name: class, dtype: int64

In [20]:
qald_train = pd.read_csv("../data/QALD/QALD-train.csv", sep=';')
qald_test = pd.read_csv("../data/QALD/QALD-test.csv", sep=';')

qald = qald_train.append(qald_test)
qald = qald[qald['class'] == 'BOOL']

In [21]:
qald['class'].value_counts()

BOOL    41
Name: class, dtype: int64

In [22]:
boolean_questions = list(lc_quad.question.values) + list(qald.question.values)

In [23]:
len(boolean_questions)

409

In [24]:
forward_questions = list()
backward_questions = list()

for question in train['Questions']:
    if question['PredicateList'][0]['Direction'] == 'backward':
        backward_questions.append(question['Query'])
    elif question['PredicateList'][0]['Direction'] == 'forward':
        forward_questions.append(question['Query'])
        
fwd_bwd_questions = random.sample(forward_questions + backward_questions, 500)

In [25]:
len(fwd_bwd_questions)

500

In [26]:
X = distance_questions + boolean_questions + fwd_bwd_questions
y = ['distance' for i in range(len(distance_questions))] + ['boolean' for i in range(len(boolean_questions))] + ['FWD_BWD' for i in range(len(fwd_bwd_questions))]

In [27]:
dataframe = pd.DataFrame.from_dict({'question': X, 'class': y})

In [32]:
dataframe['clean text'] = dataframe['question'].apply(lambda x: preprocess_text(x))

In [44]:
vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,1))
vectorizer.fit(dataframe['clean text'])

pickle.dump(vectorizer, open("template_vectorizer.model", 'wb'))

NameError: name 'TfidfVectorizer' is not defined

In [45]:
encoder = LabelEncoder()
encoder.fit(dataframe['class'])

pickle.dump(encoder, open("template_encoder.model", 'wb'))

NameError: name 'LabelEncoder' is not defined

In [33]:
print("Total number of unique n_grams in data = {0}".format(len(list(vectorizer.vocabulary_.keys()))))

NameError: name 'vectorizer' is not defined

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(dataframe['clean text'], dataframe['class'], test_size=0.33, random_state=42)

In [37]:
X_train.shape

(810,)

In [38]:
X_test.shape

(399,)

In [48]:
X_train = vectorizer.transform(X_train)
y_train = encoder.transform(y_train)

NameError: name 'vectorizer' is not defined

In [234]:
classifier = LogisticRegression(random_state=seed).fit(X_train, y_train)
pickle.dump(classifier, open("template_classifier.model", 'wb'))

In [235]:
X_test = vectorizer.transform(X_test)
y_test = encoder.transform(y_test)

In [236]:
y_pred = classifier.predict(X_test)

In [237]:
print(classification_report(y_test, y_pred,target_names = encoder.inverse_transform([0, 1, 2, ])))

             precision    recall  f1-score   support

    FWD_BWD       0.98      1.00      0.99       158
    boolean       1.00      0.97      0.99       141
   distance       1.00      1.00      1.00       100

avg / total       0.99      0.99      0.99       399



/home/alex/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [238]:
text = preprocess_text("was angela merkel born in hamburg")
print(text)
vector = vectorizer.transform([text])
prediction = classifier.predict(vector)
probas = classifier.predict_proba(vector)
class_ = encoder.inverse_transform(prediction)

print(class_)

was angela merkel born in hamburg
['boolean']


/home/alex/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [176]:
probas

array([[0.6223405 , 0.26254598, 0.11511353]])

In [177]:
max(probas[0])

0.6223404984596098

## FWD-BWD Classifier

In [39]:
X = forward_questions + backward_questions
y = ['forward' for i in range(len(forward_questions))] + ['backward' for i in range(len(backward_questions))]

In [40]:
dataframe = pd.DataFrame.from_dict({'question': X, 'class': y})

In [41]:
dataframe['clean text'] = dataframe['question'].apply(lambda x: preprocess_text(x))

In [181]:
vectorizer = TfidfVectorizer(min_df=3,ngram_range=(1,1))
vectorizer.fit(dataframe['clean text'])

pickle.dump(vectorizer, open("fwd_bwd_vectorizer.model", 'wb'))

In [182]:
encoder = LabelEncoder()
encoder.fit(dataframe['class'])

pickle.dump(encoder, open("fwd_bwd_encoder.model", 'wb'))

In [42]:
X_train, X_test, y_train, y_test = train_test_split(dataframe['clean text'], dataframe['class'], test_size=0.33, random_state=42)

In [43]:
X_train.shape

(20224,)

In [44]:
X_test.shape

(9962,)

In [184]:
X_train = vectorizer.transform(X_train)
y_train = encoder.transform(y_train)

In [185]:
classifier = LogisticRegression(random_state=seed).fit(X_train, y_train)
pickle.dump(classifier, open("fwd_bwd_classifier.model", 'wb'))

In [186]:
X_test = vectorizer.transform(X_test)
y_test = encoder.transform(y_test)

In [187]:
y_pred = classifier.predict(X_test)

In [188]:
print(classification_report(y_test, y_pred,target_names = encoder.inverse_transform([0, 1, ])))

             precision    recall  f1-score   support

   backward       0.93      0.88      0.91      3366
    forward       0.94      0.97      0.95      6596

avg / total       0.94      0.94      0.94      9962



/home/alex/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
